# LDA Topics with *impresso-pipelines* Package

<a target="_blank" href="https://colab.research.google.com/github/impresso/impresso-datalab-notebooks/blob/main/annotate/ldatopics_pipeline_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## What is this notebook about?
This notebook introduces the topic modeling component of the [impresso-pipelines](https://pypi.org/project/impresso-pipelines/) Python package. The broader goal of the impresso pipelines is to make the internal data processing workflows of the impresso project reusable and accessible to others. It allows external users—such as researchers, developers, or digital humanities practitioners—to apply the same processing steps we used on our historical newspaper collections to their own text collections.

The package is designed to minimize the coding effort required. By offering ready-to-use pipelines, users can adopt impresso approach to document processing with minimal configuration. This ensures consistency, comparability, and transparency in how topics are created from text document.

In this notebook, we focus on the [ldatopics](https://github.com/impresso/impresso-pipelines/tree/main/impresso_pipelines/ldatopics) subpackage, which enables topic modeling using the Impresso MALLET inference pipeline. You will learn how to apply the LDA Topics pipeline to your text data, explore different usage options, and interpret the results.


## Why is this useful?
- Automates the full Impresso workflow—language identification, lemmatization and topic inference—in a single call.  
- Supports multiple languages (e.g., Luxemburgish, French, German) with the corresponding spaCy model.  
- Leverages versioned, pre-trained MALLET models to guarantee reproducible and comparable results.  
- Returns structured JSON (`uid`, `ts`, `lg`, `topic-count`, `topics`) ready for downstream analysis.
    - `uid`: Unique content item id
    - `ts`: Time stamp
    - `lg`: Language
    - `topic-count`:
    - `topics`: 
- Enables rapid thematic exploration of large text collections with minimal setup and configuration.  


## How it works
The pipeline assigns your text the most relevant topics in four internal processing steps:

- **Language identifiaction**: Auto-detects the text language (e.g. `lg: 'fr'`, `lg: 'lb'`).  
- **Lemmatization**: Loads the corresponding spaCy model to tokenize, part-of-speech tag and normalize words into lemmas.  
- **Topic inference**: Feeds the lemmas of nouns and proper nouns into MALLET’s inference pipeline to compute the contribution of each earlier pre-computed topic.  
- **JSON formatting**: Returns a JSON object with `uid`, `ts`, `lg`, `topic-count`, and a list of topics above `min_relevance`, each with an ID (`t`) and relevance (`relevance`).




## What will you learn?

In this notebook, you will:

- Understand the `ldatopics_pipeline` function and its core steps.  
- See how the pipeline auto-detects text language with Impresso’s **langident**.  
- Learn how spaCy tokenizes and lemmatizes your text for modelling.  
- Run topic inference using MALLET’s pre-trained models.  
- Customize your output IDs with the `doc_name` parameter.  
- Interpret the JSON output (`uid`, `ts`, `lg`, `topic-count`, `topics`) to extract key themes.  
<!-- - Explore advanced usage, including overriding defaults and tweaking the `min_p` threshold. -->

By the end of this notebook, you’ll be able to confidently apply and interpret topic modelling with the Impresso MALLET pipeline.

## Useful resources
- For technical details on this library, please refer to the repository of the [Impresso Pipelines package](https://github.com/impresso/impresso-pipelines/tree/main).

## Prerequisites
First, start by installing `impresso-pipelines` package. Please note that if you are using Google Colab, you will need to restart the kernel after installing the package to apply changes. To do so, go to *Runtime* and select *Restart session*.



In [ ]:
%pip install "impresso_pipelines[ldatopics]"

## Basic Usage

Start by importing the necessary module from `impresso-pipelines` package


In [ ]:
from impresso_pipelines.ldatopics import LDATopicsPipeline

ldatopics_pipeline = LDATopicsPipeline()

Once you initialize the pipeline, you can simply provide the text you'd like to assign topics to. Let's infer the topics of a historical French news article.


In [3]:
text = """La vie dans un petit village est paisible et rythmée par les saisons.
Chaque matin, les habitants se saluent en se croisant dans les rues étroites,
et l’odeur du pain frais s’échappant de la boulangerie emplit l’air. Les enfants
vont à l’école à pied, les commerçants ouvrent leurs boutiques, et les anciens
s’assoient sur les bancs pour discuter de la pluie et du beau temps. Ici, tout
le monde se connaît, et l’entraide fait partie du quotidien."""

First time you use the pipeline it will download the necessary models for the specific language. Please be patient — subsequent runs will be much faster.

In [ ]:
ldatopics_pipeline(text)

If no language is explicitly specified, the LDA Topics pipeline uses the Impresso
`langident` pipeline to automatically detect the language of the text. For more details
on the `langident` pipeline, please refer to the [langident
pipeline demo notebook](https://github.com/impresso/impresso-datalab-notebooks/tree/main/annotate/langident_pipeline_demo.ipynb).

After detecting the document’s language, the pipeline internally applies the corresponding spaCy model to tokenize, part-of-speach tag and lemmatize the text. For interpretability and efficiency reasons we only consider the nouns. The filtered lemmas are then fed into a MALLET inference component. You can find these pre-packaged models on [Impresso Hugging Face](https://huggingface.co/impresso-project/mallet-topic-inferencer/tree/main/models/tm).

**Interpretation of the result:**

1. **Language & metadata**  
   - `lg: 'fr'` tells you the pipeline detected French.  
   - `ts` is the timestamp of the inference. This makes it easy to distinguish and locate the output you’re interested in when you execute the pipeline multiple times.
   - `uid` is your Unique document ID:  
     • If you don’t supply a name, the pipeline auto-assigns “doc1”, “doc2”, … based on how many docs you’ve processed so far.  
     • You can override this by passing a custom `name` parameter when calling the pipeline - see the “Advanced Usage” section of this notebook for an example.

2. **Total topics in the model**  
   - `topic-count: 100` means the underlying French model contains 100 topics.

3. **Which topics actually showed up**  
   - Only topics ≥ `min_relevance` (2%) are reported. Here you see nine topics above that cutoff. Relevance (probability) values will never add up to 100% (1.0) because there are always some topics that were assigned but are very insignificant and that will never be shown.

4. **Top topic = tm-fr-all-v2.0_tp04_fr (16%)**  
   - This is the single strongest theme in your document.  
   - A weight of 0.16 means 16 % of the inferred topic-mixture is assigned to topic 04.  
   - To see what “topic 04” actually _is_, follow the `topic_model_description` link in the output and look up the keywords/label for `_tp04_fr`.

5. **Runner-ups**  
   - Topic 22 at 11.7 %  
   - Topic 06 at 10 %  
   - Topic 36 at 8.3 %  
   - …down to the ninth topic at 2.3 %.

Taken together, these nine topics explain the main themes of your French text. You can sum their probabilities (here about 56 %) to see how much of the document is “covered,” and inspect each topic’s description in the JSONL file to attach human-readable labels.

## Advanced Usage


This pipeline offers several additional attributes that can be used when calling it to manipulate or gain a deeper understanding of the results and include the corresponding metadata from the processing pipeline for documentation. These attributes include `language`, `doc_name`:

- `language`: Accepts language abbreviation strings such as "fr" (French) or "de" (German). If provided, the pipeline assumes the specified language and skips the language detection step, directly using the corresponding spaCy and Mallet models. Currently supported languages are - French, German, Luxembourgish.

- `doc_name`: A string identifier with no spaces (e.g., "my_document"). When you provide this, it replaces the autogenerated document ID. The internal counter only increments for calls where you don’t specify a name, so the next unnamed document will pick up from the following number.

- `min_relevance`: Specify `min_relevance` as a floating-point number (for example, `0.05`). The minimum (and default) value is `0.02`; supplying any value below 0.02 will cause the pipeline to raise an error.

- `diagnostics_lemmatization`: When set to `True`, the pipeline will display the lemmatized tokens that are fed into the topic model. By default this is `False`.

- `diagnostics_topics`: When set to `True`, the pipeline will return the top 10 most significant words for each identified topic, making it easy to understand what each topic represents. By default this is `False`. For a deeper explanation, see the `topic_model_description` file included in the output.



**Example 1**: `language`


In [ ]:
# Using the same French text example as before
ldatopics_pipeline(text, language="lb")

Even though the provided text is clearly in French, specifying the language as Luxembourgish, for example, forces the pipeline to use the corresponding spaCy and MALLET models for that language. If the selected language is unsupported, the pipeline will return an appropriate error message.


**Example 2**: `doc_name`


In [ ]:
# Using the same French text example as before
ldatopics_pipeline(text, doc_name="my_document")

As you can see, the `uid` is now "my_document" as specified, making it much easier to manage your outputs when you run the ldatopics pipeline multiple times or save them in an external file.

**Example 3:** `min_relevance`

In [ ]:
ldatopics_pipeline(
    text,
    min_relevance=0.05
)

By changing the `min_relevance` threshold from 0.02 to 0.05, the pipeline filters out any topic whose assignment relevance (probability) falls below 5 %. As a result, only the most confidently detected topics (those at or above 0.05) are returned, so you see fewer—but stronger—topic assignments.

**Example 4:** `diagnostics_lemmatization`

In [ ]:
ldatopics_pipeline(
    text,
    diagnostics_lemmatization=True
)

Enabling `diagnostics_lemmatization` adds a list of the lemmatized tokens that the topic model actually sees. By examining those, you will instantly know which words are actually used to assign topics to your document.

**Example 5:** `diagnostics_topics`

In [ ]:
ldatopics_pipeline(
    text,
    diagnostics_topics=True
)

When `diagnostics_topics` is enabled, the output includes each detected topic along with its top 10 most significant words, giving you a clear snapshot of what each topic represents.

**Example 6**: All at once

In [ ]:
# Using the same French text example as before
ldatopics_pipeline(
    text,
    language="fr",
    doc_name="my_document",
    min_relevance=0.05,
    diagnostics_lemmatization=True,
    diagnostics_topics=True
)

You can use a mix of additional parameters, or all of them at once, to deeper controll the output of your LDA Topics. In the example above, we set the language to French and name of the document to "my_document".


## Limitations of the LDA Topics Pipeline


In [11]:
short_de_text = "Alex geht ins Büro."

In [ ]:
# Example 1: Very short text
ldatopics_pipeline(short_de_text, diagnostics_lemmatization=True)

As shown, LDA Topics pipeline struggles with **short texts**, leading to very flat and probably unreleable distribution. Additionally, anything that is **not** **NOUN** or **PRONOUN** does not contribute to topic modeling due to lemmatization, making the short text even shorter. Starting from 10 nouns it starts getting meaningful.


## Conclusion
In this notebook, we’ve explored the LDA Topics Pipeline from the Impresso package, which transforms raw text into topic distributions using language detection, spaCy lemmatization, and MALLET inference. The pipeline:

- **Automatically detects** text language via langident (or accepts a manual override)  
- **Loads** the matching spaCy model to tokenize and lemmatize your text  
- **Runs** MALLET’s inference pipeline with versioned, pre-trained topic models  
- **Returns** a structured JSON (`uid`, `ts`, `lg`, `topic-count`, `topics`) ready for analysis  
- **Supports** custom document IDs (`doc_name`)
- **Has** limitations with very short texts or little amount of nouns and pronouns
<!-- and adjustable reporting thresholds (`min_p`)   -->

The pipeline provides a reproducible, multi-lingual approach to topic modelling—ideal for thematic exploration of large text collections and cross-language comparisons.  


## Next steps


Since this subpackage relies on the Impresso langident subpackage, you might be interested in exploring the [demo notebook](https://github.com/impresso/impresso-datalab-notebooks/tree/main/annotate/langident_pipeline_demo.ipynb) for langident.

Additionally, you can find more technical details in the repository of the [Impresso Pipelines package](https://github.com/impresso/impresso-pipelines/tree/mallet_pipeline).


---
## Project and License info

### Notebook credits [CreditLogo.png](https://credit.niso.org/)

INSERT CREDITS HERE
<br></br>
This notebook is published under [CC BY 4.0 License](https://creativecommons.org/licenses/by/4.0/)
<br><a target="_blank" href="https://creativecommons.org/licenses/by/4.0/">
  <img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by.png"  width="100" alt="Open In Colab"/>
</a>
<br></br>

### Impresso project

[Impresso - Media Monitoring of the Past](https://impresso-project.ch) is an interdisciplinary research project that aims to develop and consolidate tools for processing and exploring large collections of media archives across modalities, time, languages and national borders. The first project (2017-2021) was funded by the Swiss National Science Foundation under grant No. [CRSII5_173719](http://p3.snf.ch/project-173719) and the second project (2023-2027) by the SNSF under grant No. [CRSII5_213585](https://data.snf.ch/grants/grant/213585) and the Luxembourg National Research Fund under grant No. 17498891.
<br></br>
### License

All Impresso code is published open source under the [GNU Affero General Public License](https://github.com/impresso/impresso-pyindexation/blob/master/LICENSE) v3 or later.


---

<p align="center">
  <img src="https://github.com/impresso/impresso.github.io/blob/master/assets/images/3x1--Yellow-Impresso-Black-on-White--transparent.png?raw=true" width="350" alt="Impresso Project Logo"/>
</p>

